In [ ]:
%pip install cassandra-driver
%pip install pandas


In [ ]:
import pandas as pd

data = pd.read_csv("/home/olha/olha-de-internship/_nosql/Student Depression Dataset.csv")
print(data.dtypes)


In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
auth_provider = PlainTextAuthProvider(username='admin', password='admin')
cluster = Cluster(['localhost'], port=9042, auth_provider=auth_provider)
session = cluster.connect()

def find_keyspaces():
    key_spaces = session.execute('DESC KEYSPACES')
    for key_space in key_spaces:
        print(key_space)
        
find_keyspaces()        

In [ ]:
create_keyspace_query = '''
    CREATE KEYSPACE studentdeprsn WITH REPLICATION = {'class' : 'SimpleStrategy','replication_factor' : 1};
'''

session.execute(create_keyspace_query)

find_keyspaces()

In [ ]:
session.execute('USE studentdeprsn')


In [ ]:
create_table_query = '''
CREATE TABLE student_depression (
    city text,
    gender text,
    id int,
    age int,
    gpa float,
    sleep_duration int,
    profession text,
    work_pressure int,
    academic_pressure int,
    study_satisfaction int,
    job_satisfaction int,
    dietary_habits text,
    loaded_at timestamp,
    PRIMARY KEY ((city, gender), id)
)
'''
session.execute(create_table_query)


for table in session.execute('DESC TABLES'):
    print(table)

In [ ]:
from datetime import datetime
insert_query = '''
    INSERT INTO student_depression (city, gender, id, age, gpa, sleep_duration, profession,work_pressure,academic_pressure, study_satisfaction, job_satisfaction, dieatary_habits, loaded_at )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

for _, row in data.iterrows():
    session.execute(insert_query, (
        row['City'],                 
        row['Gender'],               
        row['id'],                
        row['Age'],
        row['CGPA'],
        row["Sleep Duration"],
        row['Profession'],
        row["Work Pressure"],
        row["Academic Pressure"],
        row["Study Satisfaction"],
        row["Job Satisfaction"],
        row["Dietary Habits"],
        datetime.now()               
    ))

In [ ]:

create_new_table_query = '''
    CREATE TABLE student_depression_new (
        id int PRIMARY KEY,
        gender text,
        age float,  
        city text,
        gpa float,
        sleep_duration text,
        profession text,
        work_pressure float,
        academic_pressure float,
        study_satisfaction float,
        job_satisfaction float,
        dietary_habits text,
        loaded_at timestamp
    )
    
'''
session.execute(create_new_table_query)

In [ ]:
from datetime import datetime


insert_data = '''
    INSERT INTO student_depression_new (
        id, gender, age, city, gpa, sleep_duration, profession, 
        work_pressure, academic_pressure, study_satisfaction, 
        job_satisfaction, dietary_habits, loaded_at
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

truncate_data = '''
    TRUNCATE TABLE student_depression_new
'''
session.execute(truncate_data)

for _, row in data.iterrows():
    session.execute(insert_data, (
        row['id'],
        row['Gender'],
        float(row['Age']),
        row['City'],
        float(row['CGPA']),
        row["Sleep Duration"],
        row['Profession'],
        float(row["Work Pressure"]),
        float(row["Academic Pressure"]),
        float(row["Study Satisfaction"]),
        float(row["Job Satisfaction"]),
        row["Dietary Habits"],
        datetime.now()  
    ))




### Read

In [ ]:
def read_all_records(table_name):
    try:
        return session.execute("SELECT * FROM {}".format(table_name))
    except Exception as e:
        return "Read fails with " + e

def print_records(records):
    for record in records:
        print(record)

records = read_all_records('student_depression_new')

print_records(records)

### Filter

In [ ]:
def read_records_by_city(table_name, city):
    try:
        return session.execute("SELECT * FROM {} WHERE city='{}' ALLOW FILTERING".format(table_name, city))
    except Exception as e:
        return "Read fails with " + e

print_records(read_records_by_city('student_depression_new', 'Agra'))

### Update

In [ ]:
def update_record(table_name, id, updated_response_value):
    try:
        return session.execute("UPDATE {} SET age={} WHERE id={}".format(table_name, updated_response_value, id))
    except Exception as e:
        return f"UPDATE fails with: {str(e)}"

update_record('student_depression_new', '12888', '35')

### Delete

In [ ]:
def delete_records_by_id(table_name, id):
    try:
        return session.execute("DELETE FROM {} WHERE id={}".format(table_name, id))
    except Exception as e:
        return f"DELETE fails with: {str(e)}"

delete_records_by_id('student_depression_new', '12888')